<a href="https://www.kaggle.com/code/aabdollahii/gradio-user-interface?scriptVersionId=282424374" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import torch
import math
import gradio as gr
import spacy
import pandas as pd
import numpy as np
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from collections import Counter


In [ ]:
# --- MODEL SETUP ---
# We force CPU if CUDA is acting weird, but usually try CUDA first
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"⏳ Loading models on {device}...")

try:
    # Load GPT-2 for Perplexity
    model_id = "gpt2"
    tokenizer = GPT2TokenizerFast.from_pretrained(model_id)
    model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
    model.eval()

    # Load spaCy for POS
    nlp = spacy.load("en_core_web_sm")
    print("✅ Models loaded successfully.")
except Exception as e:
    print(f"❌ Error loading models: {e}")
    print("Try restarting the session again if this persists.")

In [ ]:
# --- FUNCTIONS ---

def calculate_perplexity(text):
    if not text or len(text.strip()) == 0:
        return 0
        
    encodings = tokenizer(text, return_tensors="pt")
    max_length = model.config.n_positions
    stride = 512
    seq_len = encodings.input_ids.size(1)

    nlls = []
    prev_end_loc = 0
    
    for begin_loc in range(0, seq_len, stride):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc 
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            neg_log_likelihood = outputs.loss

        nlls.append(neg_log_likelihood)
        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    if not nlls: return 0
    ppl = torch.exp(torch.stack(nlls).mean())
    return ppl.item()

def analyze_pos_features(text):
    doc = nlp(text)
    total_tokens = len(doc)
    if total_tokens == 0: return {}, 0
    
    counts = Counter([token.pos_ for token in doc])
    
    ratios = {
        "Noun Ratio": counts.get("NOUN", 0) / total_tokens,
        "Verb Ratio": counts.get("VERB", 0) / total_tokens,
        "Adjective Ratio": counts.get("ADJ", 0) / total_tokens,
        "Pronoun Ratio": counts.get("PRON", 0) / total_tokens,
    }
    
    probs = np.array([c / total_tokens for c in counts.values()])
    entropy = -(probs * np.log2(probs)).sum()
    
    return ratios, entropy

def detect_ai_human(text):
    if not text.strip():
        return "Please enter text.", {}, {}, "N/A"

    ppl = calculate_perplexity(text)
    ratios, entropy = analyze_pos_features(text)
    
    # Heuristic Scoring
    ai_score = 0
    human_score = 0
    
    # Perplexity Logic
    if ppl < 20: ai_score += 4
    elif ppl < 30: ai_score += 2
    elif ppl > 45: human_score += 4
    elif ppl > 35: human_score += 2
    
    # Entropy Logic
    if entropy < 3.32: ai_score += 1
    elif entropy > 3.36: human_score += 1
    
    # Ratio Logic
    if ratios["Noun Ratio"] > 0.22: ai_score += 1
    if ratios["Verb Ratio"] > 0.12: human_score += 1
    
    if ai_score > human_score: verdict = "Likely AI "
    elif human_score > ai_score: verdict = "Likely Human "
    else: verdict = "Uncertain "

    return verdict, f"{ppl:.2f}", f"{entropy:.4f}", ratios

In [ ]:
# --- CORRECTED GRADIO UI CODE ---
import gradio as gr

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🕵️ AI vs Human Text Analyzer")
    gr.Markdown("Server running on Kaggle T4 GPU.")
    
    with gr.Row():
        with gr.Column():
            input_text = gr.Textbox(label="Input Text", lines=8, placeholder="Type here...")
            analyze_btn = gr.Button("Analyze", variant="primary")
        
        with gr.Column():
            lbl_verdict = gr.Label(label="Verdict")
            with gr.Row():
                # We define the label variable here as lbl_ppl
                lbl_ppl = gr.Textbox(label="Perplexity") 
                lbl_entropy = gr.Textbox(label="Entropy")
            lbl_ratios = gr.JSON(label="POS Ratios")

    analyze_btn.click(
        fn=detect_ai_human,
        inputs=input_text,
        # CORRECTED LINE BELOW: used 'lbl_ppl' instead of 'ppl'
        outputs=[lbl_verdict, lbl_ppl, lbl_entropy, lbl_ratios]
    )

# Launch with server share enabled
demo.launch(share=True, debug=True)


-----

In [1]:
import torch
import math
import gradio as gr
import spacy
import pandas as pd
import numpy as np
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, BertTokenizer, BertModel
from collections import Counter
import torch.nn as nn
import re
import unicodedata

2025-11-28 09:40:57.231397: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764322857.461171     112 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764322857.524001     112 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# --- MODEL SETUP ---
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"⏳ Loading models on {device}...")

try:
    # Load GPT-2 for Perplexity
    model_id = "gpt2"
    gpt2_tokenizer = GPT2TokenizerFast.from_pretrained(model_id)
    gpt2_model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
    gpt2_model.eval()

    # Load spaCy for POS
    nlp = spacy.load("en_core_web_sm")
    
    # Load BERT tokenizer and define model class
    bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
    
    class BERTClassifier(nn.Module):
        def __init__(self, model_name):
            super(BERTClassifier, self).__init__()
            self.bert = BertModel.from_pretrained(model_name)
            self.drop = nn.Dropout(p=0.3)
            self.out = nn.Linear(in_features=768, out_features=2)
            
        def forward(self, input_ids, attention_mask):
            output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
            pooled_output = output[1]
            output = self.drop(pooled_output)
            return self.out(output)
    
    # Initialize BERT model and load trained weights
    bert_model = BERTClassifier('bert-base-cased').to(device)
    bert_model.load_state_dict(torch.load('/kaggle/input/aigen-bertmodel/pytorch/default/1/bert_model_state.bin', map_location=device))
    bert_model.eval()
    
    print("All models loaded successfully.")
except Exception as e:
    print(f" Error loading models: {e}")
    print("Try restarting the session if this persists.")


⏳ Loading models on cpu...


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

All models loaded successfully.


In [3]:
# --- TEXT PREPROCESSING ---
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    
    text = unicodedata.normalize('NFKC', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[\r\n\t]+', ' ', text)
    text = text.replace('\\', '')
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [4]:
# --- BERT PREDICTION ---
def predict_with_bert(text, max_len=512):
    """Returns probability that text is AI-generated"""
    if not text.strip():
        return 0.5, "No text provided"
    
    cleaned_text = preprocess_text(text)
    
    encoding = bert_tokenizer.encode_plus(
        cleaned_text,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
        probs = torch.softmax(outputs, dim=1)
        ai_prob = probs[0][1].item()  # Probability of AI class
        
    prediction = "AI-generated" if ai_prob > 0.5 else "Human-written"
    confidence = max(ai_prob, 1 - ai_prob) * 100
    
    return ai_prob, f"{prediction} ({confidence:.1f}% confidence)"


In [5]:
# --- PERPLEXITY CALCULATION ---
def calculate_perplexity(text):
    if not text or len(text.strip()) == 0:
        return 0
        
    encodings = gpt2_tokenizer(text, return_tensors="pt")
    max_length = gpt2_model.config.n_positions
    stride = 512
    seq_len = encodings.input_ids.size(1)

    nlls = []
    prev_end_loc = 0
    
    for begin_loc in range(0, seq_len, stride):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc 
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = gpt2_model(input_ids, labels=target_ids)
            neg_log_likelihood = outputs.loss

        nlls.append(neg_log_likelihood)
        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    if not nlls: 
        return 0
    ppl = torch.exp(torch.stack(nlls).mean())
    return ppl.item()

In [6]:
# --- POS ANALYSIS ---
def analyze_pos_features(text):
    doc = nlp(text)
    total_tokens = len(doc)
    if total_tokens == 0: 
        return {}, 0
    
    counts = Counter([token.pos_ for token in doc])
    
    ratios = {
        "Noun Ratio": counts.get("NOUN", 0) / total_tokens,
        "Verb Ratio": counts.get("VERB", 0) / total_tokens,
        "Adjective Ratio": counts.get("ADJ", 0) / total_tokens,
        "Pronoun Ratio": counts.get("PRON", 0) / total_tokens,
    }
    
    probs = np.array([c / total_tokens for c in counts.values()])
    entropy = -(probs * np.log2(probs)).sum()
    
    return ratios, entropy

In [7]:
def detect_ai_human(text):
    if not text.strip():
        return "Please enter text.", {}, {}, "N/A", "N/A", "N/A"

    # BERT prediction
    ai_prob, bert_verdict = predict_with_bert(text)
    
    # Perplexity
    ppl = calculate_perplexity(text)
    
    # POS analysis
    ratios, entropy = analyze_pos_features(text)
    
    # Heuristic scoring (from original code)
    ai_score = 0
    human_score = 0
    
    if ppl < 20: 
        ai_score += 4
    elif ppl < 30: 
        ai_score += 2
    elif ppl > 45: 
        human_score += 4
    elif ppl > 35: 
        human_score += 2
    
    if entropy < 3.32: 
        ai_score += 1
    elif entropy > 3.36: 
        human_score += 1
    
    if ratios["Noun Ratio"] > 0.22: 
        ai_score += 1
    if ratios["Verb Ratio"] > 0.12: 
        human_score += 1
    
    if ai_score > human_score: 
        heuristic_verdict = "Likely AI"
    elif human_score > ai_score: 
        heuristic_verdict = "Likely Human"
    else: 
        heuristic_verdict = "Uncertain"

    return bert_verdict, heuristic_verdict, f"{ppl:.2f}", f"{entropy:.4f}", f"{ai_prob:.4f}", ratios

In [14]:
# --- GRADIO UI ---
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🕵️ AI vs Human Text Analyzer")
    gr.Markdown("Combines BERT classifier with perplexity and linguistic analysis. Running on Kaggle T4 GPU.")
    
    with gr.Row():
        with gr.Column():
            input_text = gr.Textbox(label="Input Text", lines=10, placeholder="Paste text here for analysis...")
            analyze_btn = gr.Button("🔍 Analyze Text", variant="primary", size="lg")
        
        with gr.Column():
            gr.Markdown("### 🤖 BERT Model Prediction")
            lbl_bert = gr.Textbox(label="BERT Verdict", interactive=False)
            lbl_bert_prob = gr.Textbox(label="AI Probability", interactive=False)
            
            gr.Markdown("### 📊 Heuristic Analysis")
            lbl_heuristic = gr.Textbox(label="Heuristic Verdict", interactive=False)
            
            with gr.Row():
                lbl_ppl = gr.Textbox(label="Perplexity", interactive=False)
                lbl_entropy = gr.Textbox(label="POS Entropy", interactive=False)
            
            lbl_ratios = gr.JSON(label="Part-of-Speech Ratios")

    analyze_btn.click(
        fn=detect_ai_human,
        inputs=input_text,
        outputs=[lbl_bert, lbl_heuristic, lbl_ppl, lbl_entropy, lbl_bert_prob, lbl_ratios]
    )

# Launch
demo.launch(share=True, debug=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://a91341d9b86c3a571e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a91341d9b86c3a571e.gradio.live


In [13]:
# Add this debug function to check what your model actually learned
def debug_bert_predictions(text):
    cleaned_text = preprocess_text(text)
    
    encoding = bert_tokenizer.encode_plus(
        cleaned_text,
        add_special_tokens=True,
        max_length=512,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
        probs = torch.softmax(outputs, dim=1)
        
        print(f"Raw logits: {outputs[0]}")
        print(f"Class 0 prob: {probs[0][0].item():.4f}")
        print(f"Class 1 prob: {probs[0][1].item():.4f}")
        
    return probs

# Test with known human text
debug_bert_predictions("HIIIIIIIIIIIIII Its meeeeeeeeeee")
# Test with known AI text  
debug_bert_predictions("The implementation of artificial intelligence systems requires careful consideration.")


Raw logits: tensor([-3.6124,  3.6595])
Class 0 prob: 0.0007
Class 1 prob: 0.9993
Raw logits: tensor([-6.0712,  6.2176])
Class 0 prob: 0.0000
Class 1 prob: 1.0000


tensor([[4.6031e-06, 1.0000e+00]])

In [12]:
# After loading, verify the weights aren't just random
print("First layer weight sample:", bert_model.bert.embeddings.word_embeddings.weight[0][:5])


First layer weight sample: tensor([-0.0005, -0.0415,  0.0131,  0.0058, -0.0376], grad_fn=<SliceBackward0>)


In [15]:
# Add detailed logging to compare preprocessing outputs
def debug_preprocessing(text):
    print(f"Original text length: {len(text)}")
    print(f"Original first 100 chars: {text[:100]}")
    
    cleaned = preprocess_text(text)
    print(f"Cleaned text length: {len(cleaned)}")
    print(f"Cleaned first 100 chars: {cleaned[:100]}")
    
    encoding = bert_tokenizer.encode_plus(
        cleaned,
        add_special_tokens=True,
        max_length=512,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    
    print(f"Token IDs first 20: {encoding['input_ids'][0][:20]}")
    print(f"Attention mask sum: {encoding['attention_mask'].sum()}")
    
    return encoding

# Test with a sample
test_text = "This is a test sentence to check preprocessing."
debug_preprocessing(test_text)


Original text length: 47
Original first 100 chars: This is a test sentence to check preprocessing.
Cleaned text length: 47
Cleaned first 100 chars: This is a test sentence to check preprocessing.
Token IDs first 20: tensor([  101,  1188,  1110,   170,  2774,  5650,  1106,  4031,  3073,  1643,
         2180, 22371,  1158,   119,   102,     0,     0,     0,     0,     0])
Attention mask sum: 15


{'input_ids': tensor([[  101,  1188,  1110,   170,  2774,  5650,  1106,  4031,  3073,  1643,
          2180, 22371,  1158,   119,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [16]:
# After loading the model
bert_model.eval()

# Disable all dropout layers explicitly
for module in bert_model.modules():
    if isinstance(module, nn.Dropout):
        module.p = 0.0
        module.training = False

# Disable gradient computation globally
torch.set_grad_enabled(False)


In [2]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
import unicodedata
import re

# Preprocessing function
def preprocess_text(text):
    text = unicodedata.normalize('NFKC', text)
    text = re.sub(r'<[^>]+>', '', text)
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Model architecture
class BERTClassifier(nn.Module):
    def __init__(self, n_classes=2):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
    
    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = output[1]
        output = self.drop(pooled_output)
        return self.out(output)

# Initialize
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load tokenizer FIRST (before any model operations)
print("Loading tokenizer...")
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# Load model
print("Loading model...")
bert_model = BERTClassifier(n_classes=2)
bert_model.load_state_dict(torch.load(
    '/kaggle/input/aigen-bertmodel/pytorch/default/1/bert_model_state.bin',
    map_location=device
))
bert_model = bert_model.to(device)
bert_model.eval()

# Disable gradients
torch.set_grad_enabled(False)

print("Model loaded successfully!\n")

# Test samples
human_text = """In the artical Car Free Cities people all over the world are going carless. In German suburbs life goes on without cars."""

ai_text = """Climate change has become a hot topic in recent years, and many people are starting to realize the importance of taking action."""

def test_sample(text, sample_name, expected_label):
    print(f"\n{'='*60}")
    print(f"Testing: {sample_name} (Expected: Label {expected_label})")
    print(f"{'='*60}")
    
    # Preprocess and tokenize
    cleaned = preprocess_text(text)
    encoding = bert_tokenizer.encode_plus(
        cleaned,
        add_special_tokens=True,
        max_length=512,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # Predict
    outputs = bert_model(input_ids, attention_mask)
    probs = torch.softmax(outputs, dim=1)
    
    class_0_prob = probs[0][0].item()
    class_1_prob = probs[0][1].item()
    predicted = torch.argmax(probs).item()
    
    print(f"Class 0 probability: {class_0_prob:.4f} ({class_0_prob*100:.2f}%)")
    print(f"Class 1 probability: {class_1_prob:.4f} ({class_1_prob*100:.2f}%)")
    print(f"Predicted class: {predicted}")
    print(f"Match: {'✓ CORRECT' if predicted == expected_label else '✗ WRONG'}")
    
    return predicted == expected_label

# Run tests
result1 = test_sample(human_text, "HUMAN (shortened)", expected_label=0)
result2 = test_sample(ai_text, "AI (shortened)", expected_label=1)

print(f"\n{'='*60}")
print(f"SUMMARY: {sum([result1, result2])}/2 correct")
print(f"{'='*60}")


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Exception ignored in: <function _xla_gc_callback at 0x793bf991c7c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Using device: cpu
Loading tokenizer...


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Loading model...


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Model loaded successfully!


Testing: HUMAN (shortened) (Expected: Label 0)
Class 0 probability: 1.0000 (100.00%)
Class 1 probability: 0.0000 (0.00%)
Predicted class: 0
Match: ✓ CORRECT

Testing: AI (shortened) (Expected: Label 1)
Class 0 probability: 0.0000 (0.00%)
Class 1 probability: 1.0000 (100.00%)
Predicted class: 1
Match: ✓ CORRECT

SUMMARY: 2/2 correct
